# Script de migracion de datos

Este programa está diseñado para procesar datos de, en el caso de los planes de mantenimiento, se tienen 2 planes de mantenimiento (Lineas de Alta Demanda LAD y Lineas de Baja Demanda LBD) descargándolos, transformándolos y guardándolos en un archivo Excel. La clase principal, GoogleSheetProcessor, maneja todo el flujo de trabajo, desde la obtención de datos hasta su procesamiento y almacenamiento. A continuación se describe el funcionamiento detallado del programa.

## Funcionalidades Principales
1. Inicialización (__init__)
La clase se inicializa con la URL de una hoja de cálculo de Google Sheets. Durante la inicialización, se extraen los identificadores del archivo y de la hoja específica, y se construye la URL para exportar la hoja en formato CSV.

### Atributos:

sheet_url: URL de la hoja de cálculo de Google Sheets.
spreadsheet_id: ID único de la hoja de cálculo.
sheet_id: ID único de la hoja dentro de la hoja de cálculo.
csv_export_url: URL para exportar la hoja en formato CSV.

### Diccionarios predefinidos:

valores: Diccionario que asocia códigos con valores numéricos. Este diccionario de valores es dependiendo las columnas de la hoja de calculo, especifico a las frecuencias 
regimen: Diccionario que asocia códigos con sus unidades de medida correspondientes.


1. Extracción de Identificadores
extract_spreadsheet_id(url): Extrae el ID de la hoja de cálculo desde la URL.
extract_sheet_id(url): Extrae el ID de la hoja específica desde la URL.
2. Construcción de la URL de Exportación
construct_csv_export_url(): Construye la URL que permite descargar la hoja de cálculo en formato CSV.
3. Descarga de la Hoja en CSV
download_csv(output_filename='temp_sheet.csv'): Descarga la hoja de cálculo en formato CSV y la guarda con un nombre de archivo especificado (por defecto, temp_sheet.csv).
4. Procesamiento de Datos
process_data(filename="temp_sheet.csv", valores="", regimen=""):
Carga el archivo CSV y realiza una serie de transformaciones y filtrados en los datos, como la conversión de valores, la asignación de unidades, y la reestructuración del DataFrame.
Comprueba que las claves de los diccionarios valores y regimen coinciden, lanzando un AssertionError si no es así.
Filtra los datos para excluir planes y mantener solo las columnas relevantes.
Realiza transformaciones en el DataFrame para preparar la información que se almacenará en Excel.
Retorna los DataFrames df_plan, df_action, df_speciality y filtered_data.
5. Guardar Resultados en Excel
save_to_excel(output_path="Salida.xlsx", filename="mix_plan.csv", valores="", regimen=""):
Llama al método process_data para obtener los DataFrames procesados.
Guarda los DataFrames resultantes en un archivo Excel con hojas separadas para acciones, planes, especialidades y actividades filtradas.
6. Funciones Auxiliares
get_unique(df: pd.DataFrame, column: str): Genera un DataFrame con valores únicos de una columna específica, ajustando los índices.
buscarIndice(df: pd.DataFrame, valor, columna='value'): Busca el índice de un valor específico en un DataFrame y lo retorna como un entero.

```mermaid
graph TD
    A[Inicio] --> B[Inicializa GoogleSheetProcessor con lad.value]
    B --> C[Lee CSV desde input/pad.csv]
    C --> D[Inicializa GoogleSheetProcessor con lbd.value]
    D --> E[Lee CSV desde input/pbd.csv]
    E --> F[Concatena los DataFrames df1 y df2]
    F --> G[Guarda el DataFrame combinado indicado en el campo salida]    
    G --> H[Guarda el Dataframe directamente en la base de datos indicada]

```

Descripción del Flujograma
* Inicio: El proceso comienza con la inicialización del primer GoogleSheetProcessor con la URL contenida en lad.value.
* Lectura del primer CSV: Se lee el archivo CSV asociado al primer DataFrame desde la ruta input/pad.csv.
* Inicialización del segundo GoogleSheetProcessor: Se inicializa el segundo objeto GoogleSheetProcessor con la URL contenida en lbd.value.
* Lectura del segundo CSV: Se lee el archivo CSV asociado al segundo DataFrame desde la ruta input/pbd.csv.
* Concatenación de DataFrames: Los dos DataFrames (df1 y df2) se combinan en uno solo mediante pd.concat.
* Guardar el DataFrame combinado: El DataFrame combinado se guarda en un archivo CSV en la ruta input/mix_plan.csv.
* Procesamiento y Almacenamiento de hoja de calculo procesada: Se puede almacenar en archivo excel o directamente en la base de datos, 


### Pantalla 1
Al presionar el boton de _Generar Archivo Excel_ se realiza la generación del archivo excel en la carpeta output, colocando el nombre del archivo excel que se encuetnra en el campo de salida.

![Pantalla 1](assets/pantalla1.png "Pantalla 1")

### Pantalla 2

En este apartado se colocan los datos de conexion de la base de datos. Al presionar el botón de _Cargar en Base de datos_ se procede a conectar y a volcar el dataframe en la base de datos indicada en los datos de usuario, password, host y base de datos de destino.

![Pantalla 2](assets/pantalla2.png "Pantalla 2")



> Nota. Exportar directamente en la base de datos aun no es posible, se requiere complementar el codigo.

In [1]:
import pandas as pd
import requests

class GoogleSheetProcessor:
    def __init__(self, sheet_url:str):
        self.sheet_url = sheet_url
        self.spreadsheet_id = self.extract_spreadsheet_id(sheet_url)
        self.sheet_id = self.extract_sheet_id(sheet_url)
        self.csv_export_url = self.construct_csv_export_url()

        # Diccionarios originales
        self.valores = {
            "D": 1, "S": 1, "M": 5, "MC": 1, "2M": 2, "T": 3, "4M": 4, "SE": 6,
            "8M": 8, "A": 1, "1.5A": 18, "2A": 2, "3A": 3, "4A": 4, "5A": 5,
            "6A": 6, "8A": 8, "10A": 10, "1000": 1000, "6000": 6000, "22500": 22500,
            "40000": 40000, "55000": 55000
        }

        self.regimen = {
            "D": 'dia', "S": 'semana', "M": 'semana', "MC": 'mes', "2M": 'mes', "T": 'mes',
            "4M": 'mes', "SE": 'mes', "8M": 'mes', "A": 'año', "1.5A": 'mes', "2A": 'año',
            "3A": 'año', "4A": 'año', "5A": 'año', "6A": 'año', "8A": 'año', "10A": 'año',
            "1000": 'horas', "6000": 'horas', "22500": 'horas', "40000": 'horas', "55000": 'horas'
        }

    def extract_spreadsheet_id(self, url):
        return url.split('/d/')[1].split('/')[0]

    def extract_sheet_id(self, url):
        return url.split('gid=')[1]

    def construct_csv_export_url(self):
        return f"https://docs.google.com/spreadsheets/d/{self.spreadsheet_id}/export?format=csv&gid={self.sheet_id}"

    def download_csv(self, output_filename='temp_sheet.csv'):
        # Descarga el archivo CSV y lo guarda temporalmente
        response = requests.get(self.csv_export_url)
        response.raise_for_status()  # Asegurarse de que la solicitud fue exitosa
        with open(output_filename, 'wb') as f:
            f.write(response.content)
        return output_filename
    def get_unique(self, df: pd.DataFrame, column: str):
        """
        Obtiene un DataFrame con valores únicos de la columna 'Column', con índices ajustados.

        Returns:
        pd.DataFrame: Un DataFrame con valores únicos de la columna 'Column' y un índice ajustado.
        """
        df[column] = df[column].str.strip()
        df = df[df[column].notnull()]
        df_unique = pd.DataFrame(df[column].unique(), columns=['value'])
        df_unique.index = df_unique.index + 1
        return df_unique


    def buscarIndice(self, df: pd.DataFrame, valor:str, columna_id='value'):
        # Verificar si el valor está en la columna_id especificada
        """
        Busca un valor en la columna especificada del DataFrame.
        Si el valor de búsqueda es nulo o si no se encuentra, retorna el mismo valor de búsqueda.
        """
        # Validar si el valor de búsqueda es nulo        
            
        if pd.isna(valor):
            return None #pd.NA    
        
        #fkc_priority tiene el valor "BAJA " con espacio al final, eliminar el ultimo espacio Se añade a la funcion buscarIndice
        valor = valor.upper().strip()

        resultado = df[df[columna_id].str.upper() == valor]    
        # Si no encuentra el valor, retornar el mismo valor
        if resultado.empty:
            return valor
        else:
            return int(resultado.index[0])


    def read_csv(self, filename="temp_sheet.csv"):
        # Lee el archivo CSV usando pandas
        self.df = pd.read_csv(filename)
        self.df.columns = self.df.loc[2, :].to_list()  # la fila 2 como fila
        self.df = self.df.loc[4:, :]   # Obtener desde la fila 4 en adelante
        return self.df
    
    def process_data_with_validation(self, df: pd.DataFrame, valores: dict):
        # Iterar sobre las claves del diccionario valores
        for col in valores.keys():
            # Verificar si la columna existe en el DataFrame
            if col in df.columns:
                # Comprobar si la columna no es booleana
                if not pd.api.types.is_bool_dtype(df[col]):
                    # Si no es booleana, intentamos convertirla
                    df[col] = df[col].apply(lambda x: True if str(x).upper() == 'TRUE' else False)
                    # df[list(valores.keys())] = df[valores.keys()].applymap(lambda x: True if x == 'TRUE' else False)
            else:
                print(f"La columna '{col}' no se encuentra en el DataFrame.")
        return df

    def process_data(self,filename = "temp_sheet.csv",valores="", regimen=""):        
        df = pd.DataFrame()
        df = pd.read_csv(filename)

        if valores == "":valores = self.valores    
        if regimen == "":regimen = self.regimen
        
        if valores.keys() != regimen.keys():
            raise AssertionError(f"Las claves no coinciden: {valores.keys()} != {regimen.keys()}")

        # Realiza el procesamiento necesario
        # Este es un lugar para incluir toda la lógica de procesamiento
        
        # Suponiendo que el procesamiento produce 'filtered_data' y otros DataFrames
        df_plan = pd.DataFrame()  # Placeholder
        df_action = pd.DataFrame()  # Placeholder
        df_speciality = pd.DataFrame()  # Placeholder
        filtered_data = pd.DataFrame()  # Placeholder        
        #print(valores)
        ## convertir a booleano
        # df[list(valores.keys())] = df[valores.keys()].applymap(lambda x: True if x == 'TRUE' else False)
        ## convertir a booleano
        df = self.process_data_with_validation(df,valores)
        #print(df.dtypes)

        # Quitar planes
        df = df[df['Tipo_plan']!= 'Plan']   # Se cambio de Tipo a Tipo_plan el 5-9-24

        # Obtener la unidades
        parametros = regimen
        df['unidad'] = df.apply(lambda row: next((parametros[key] for key in parametros.keys() if key in row and row[key] == True), None), axis=1)
        # Obtener los valores
        parametros = valores
        df['valor'] = df.apply(lambda row: next((parametros[key] for key in parametros.keys() if key in row and row[key] == True), None), axis=1)
        # Filtrar las columnas necesarias solamente
        #print("Valores unicos en unidades: ")
        #print(df['unidad'].unique())        
        # Mantener solo las columnas necesarias
        columns = ['Plan','Accion','Trabajo','Actividad','Tipo_plan','Parada','Relevancia','Especialidad','valor','unidad']
        df = df[columns]
        # Crear la nueva columna fk_activity que tendra relaciones con las actividades padre
        df['fk_activity']= None
        df['fkc_regime']= None

        # renombrar los nombres de las columnas
        nuevos_nombres = {
            'Plan': 'fk_plan',
            'Accion': 'fk_action',
            'Actividad': 'name',
            'Tipo_plan': 'fkc_activity_type',
            'Relevancia': 'fkc_priority',
            'Especialidad': 'fk_specialty',
            'valor': 'time_interval_value',
            'unidad': 'fk_periodicity_unit',
            'Parada': 'stoppage',
        }
        df.rename(columns=nuevos_nombres, inplace=True)
                # Mantener las columnas del excel en el orden indicado
        columnas_excel = ['fk_activity','fk_plan','fk_action','name','fkc_activity_type','fkc_priority','fk_specialty','fkc_regime','stoppage','time_interval_value','fk_periodicity_unit'] 

        df = df[columnas_excel]
        df_plan = self.get_unique(df,"fk_plan")
        df_action = self.get_unique(df,"fk_action")
        df_speciality = self.get_unique(df,"fk_specialty")
        df_activity_type = self.get_unique(df,"fkc_activity_type")
        df_regime = self.get_unique(df,"fkc_regime")

        # Filter the data
        #df = df_raw.copy(deep=True)
        filtered_data = df[(df['fkc_activity_type'] == 'Actividad') | (df['fkc_activity_type'] == 'Tarea')]
        
        # Add fk_activity column
        filtered_data['fk_activity'] = None

        # Buscar fk_activity para las Tareas que provienen de una Actividad
        parent_index = None
        for i, row in filtered_data.iterrows():
            if row['fkc_activity_type'] == 'Actividad':
                parent_index = i
            elif row['fkc_activity_type'] == 'Tarea':
                filtered_data.at[i, 'fk_activity'] = parent_index

        # Obtener los ids de la relacion con los otros dataframes
        filtered_data['fk_plan']= filtered_data['fk_plan'].apply(lambda x: self.buscarIndice(df_plan,x))
        filtered_data['fk_action']= filtered_data['fk_action'].apply(lambda x: self.buscarIndice(df_action,x)) 
        filtered_data['fk_specialty']= filtered_data['fk_specialty'].apply(lambda x: self.buscarIndice(df_speciality,x)) 
        valores_lecturas=['horas','ciclos']
        # Discriminar si las lecturas son horas o ciclos colocar FECHAS O LECTURAS
        filtered_data['fkc_regime'] = filtered_data['fk_periodicity_unit'].apply(lambda x: 'LECTURAS' if x in valores_lecturas else 'FECHAS')
        # Filtrar y aplicar los cambios correspondientes, mover los valores a las columnas de uso
        filtered_data.loc[filtered_data['fk_periodicity_unit'].isin(valores_lecturas), 'usage_interval_value'] = filtered_data['time_interval_value']
        filtered_data.loc[filtered_data['fk_periodicity_unit'].isin(valores_lecturas), 'fk_usage_unit'] = filtered_data['fk_periodicity_unit']

        # Colocar los valores en las columnas timer_interva_value y fk_periodicity_unit en nulo
        filtered_data.loc[filtered_data['fk_periodicity_unit'].isin(valores_lecturas), 'time_interval_value'] = None #pd.NA
        filtered_data.loc[filtered_data['fk_periodicity_unit'].isin(valores_lecturas), 'fk_periodicity_unit'] = None #pd.NA
       
        return df_plan, df_action, df_speciality, filtered_data    


    def save_to_excel(self, output_path="Salida.xlsx",filename="mix_plan.csv",valores="",regimen=""):
        df_plan, df_action, df_speciality, filtered_data = self.process_data(filename=filename,valores=valores,regimen=regimen)  
        with pd.ExcelWriter(output_path) as writer:
            df_action.to_excel(writer, sheet_name='actions')
            df_plan.to_excel(writer, sheet_name='plans')
            df_speciality.to_excel(writer, sheet_name='specialties')
            filtered_data.to_excel(writer, sheet_name='activities')

In [2]:

# Bloque de variables
descargar = False
url_alta_demanda ="https://docs.google.com/spreadsheets/d/1oUHkuKpHtuhMirNW6SvAQ4A0ns5PZs71iZ_WFXZHNn8/edit?gid=1199302294"
archivo_ad = "input/pad_actividades.csv"
url_baja_demanda = "https://docs.google.com/spreadsheets/d/1yOaSeqRBr1FW6tvFMi_Y-s4011cKBoyiWU5dTMlujrU/edit?gid=1199302294"
archivo_bd = "input/pbd_actividades.csv"
filename_actividades = "input/mix_plan_actividades.csv"

valores = {"D": 1,"S": 1,"2S": 2,"M": 5,"MC": 1,"2M": 2,"T": 3,"SE": 6,"8M": 8,"A": 1,"1.5A": 18,"2A": 2,"3A": 3,"4A": 4,"5A": 5,"6A": 6,"8A": 8,"10A": 10,"1000": 1000,"1300": 1300,"1800": 1800,"6000": 6000,"22500": 6000,"40000": 40000,"55000": 55000,"55000C": 55000}

regimen = {"D": 'dia(s)',"S": 'semana(s)',"2S": 'semana(s)',"M": 'semana(s)',"MC": 'mes(es)',"2M": 'mes(es)',"T": 'mes(es)',"SE": 'mes(es)',"8M": 'mes(es)',"A": 'año(s)',"1.5A": 'mes(es)',"2A": 'año(s)',"3A": 'año(s)',"4A": 'año(s)',"5A": 'año(s)',"6A": 'año(s)',"8A": 'año(s)',"10A": 'año(s)',"1000": 'horas',"1300": 'horas',"1800": 'horas',"6000": 'horas',"22500": 'horas',"40000": 'horas',"55000": 'horas',"55000C": 'ciclos'}

In [3]:
# Combinacion de dataframes
gs1 = GoogleSheetProcessor(url_alta_demanda)
if descargar : gs1.download_csv(archivo_ad)
df1 = gs1.read_csv(archivo_ad)

In [4]:
gs2 = GoogleSheetProcessor (url_baja_demanda)
if descargar: gs2.download_csv(archivo_bd)
df2 = gs2.read_csv(archivo_bd)

In [5]:
if ((df1.columns == df2.columns).any) :
    print("No son iguales las columnas, verificar la igualdad de columnas")

No son iguales las columnas, verificar la igualdad de columnas


In [6]:
# Realizar el merge de ambos planes en un solo dataframe
df_merged = pd.concat([df1,df2],ignore_index=True).reset_index(drop=True)
df_merged.to_csv(filename_actividades)

In [7]:
df_plan, df_action, df_speciality, df_activities = gs1.process_data(filename=filename_actividades,valores=valores,regimen=regimen)  

In [8]:
df_activities.dtypes

fk_activity              object
fk_plan                   int64
fk_action                 int64
name                     object
fkc_activity_type        object
fkc_priority             object
fk_specialty              int64
fkc_regime               object
stoppage                 object
time_interval_value     float64
fk_periodicity_unit      object
usage_interval_value    float64
fk_usage_unit            object
dtype: object

## Funciones principales (Para Base de Datos)
1. **format_dataframe**:
Descripción: Formatea un DataFrame antes de insertarlo en una base de datos, asegurándose de que tenga todas las columnas requeridas y ajustando algunas de sus propiedades (como agregar UUIDs, establecer columnas de seguimiento como created_at, updated_at, etc.).
Uso: Asegura que las columnas entre el DataFrame y la tabla de la base de datos sean consistentes.
2. **actualizar_tabla_postgres:**
Descripción: Elimina los registros de una tabla PostgreSQL, inserta nuevos datos desde un DataFrame, y reinicia la secuencia de la columna id para evitar conflictos.
Uso: Se utiliza para actualizar completamente una tabla en PostgreSQL con nuevos datos, manteniendo la consistencia de la columna id.
3. **obtener_registros:**
Descripción: Ejecuta una consulta SELECT * en una tabla PostgreSQL y devuelve los resultados en un DataFrame. Puede filtrar por columnas específicas si se le proporcionan.
Uso: Sirve para obtener los registros de una tabla en formato DataFrame.
4. **buscarIndice:**
Descripción: Busca un valor en una columna específica de un DataFrame y devuelve el índice del primer resultado encontrado. Si no se encuentra el valor, devuelve el valor de búsqueda original.
Uso: Para encontrar la posición de un valor en un DataFrame.
5. **ejecutar_query:**
Descripción: Ejecuta una consulta SQL (que puede o no devolver filas) y devuelve el resultado en un DataFrame si aplica.
Uso: Ejecuta cualquier consulta SQL genérica, devolviendo resultados si es necesario.
6. **eliminar_registros:**
Descripción: Elimina todos los registros de una tabla PostgreSQL.
Uso: Se utiliza para limpiar una tabla antes de insertar nuevos datos.
7. **update_plans_table:**
Descripción: Elimina registros de una tabla PostgreSQL, inserta nuevos datos desde un DataFrame y reinicia la secuencia de la columna id. Se enfoca en tablas relacionadas con planes.
Uso: Función similar a actualizar_tabla_postgres, diseñada específicamente para actualizar tablas de planes.


In [9]:
from sqlalchemy import create_engine, text
from sqlalchemy.engine.result import Result
import pandas as pd
import uuid
""" Para pasar a la base de datos"""


def format_dataframe(df:pd.DataFrame,tabla:str,
                     usuario='user_mantenimiento', password='pass_M4ntenimient0',host='192.168.100.56', database='db_mantenimiento_test'):
    
    # Crear el engine de SQLAlchemy
    engine = create_engine(f'postgresql://{usuario}:{password}@{host}/{database}')
    # Leer la tabla original en un DataFrame de pandas
    df_origen = pd.read_sql_query(f"SELECT * FROM {tabla}", con=engine)

    # Renombrar la columna 'value' a 'name' si existe
    if 'value' in df.columns:
        df = df.rename(columns={'value': 'name'})

    # Si existe la columna 'uuid' en la tabla original, crear esa columna
    if 'uuid' in df_origen.columns:
        df['uuid'] = [str(uuid.uuid4()) for _ in range(len(df))]

    if 'is_active' in df_origen.columns:
        df['is_active'] = True        

    # Identificar las columnas que están en df_origen pero no en df
    missing_columns = [col for col in df_origen.columns if col not in df.columns]

    # Añadir las columnas faltantes a df con valores NaN
    for col in missing_columns:
        df[col] = None #pd.NA  # O usa otro valor predeterminado si es necesario

    # Añadir columnas comunes
    df['id'] = df.index +1 if df.index[0] ==0 else df.index
    df['created_by'] = 1
    df['updated_by'] = 1
    df['created_at'] = pd.Timestamp.now()
    df['updated_at'] = pd.Timestamp.now()
    # Validar que las columnas de df y df_origen sean iguales
    columnas_df = set(df.columns)
    columnas_df_origen = set(df_origen.columns)
    
    # Si las columnas no son iguales, lanzar un error
    assert columnas_df == columnas_df_origen, f"Las columnas no coinciden. Columnas faltantes: {columnas_df_origen - columnas_df} en df y {columnas_df - columnas_df_origen} en df_origen"        
        
    return df


def actualizar_tabla_postgres(df: pd.DataFrame, tabla: str, columna_id: str,
                              usuario='user_mantenimiento', password='pass_M4ntenimient0',host='192.168.100.56', database='db_mantenimiento_test'):    
    # Reemplazar NaN por None (que en SQL es equivalente a NULL)
    df = df.where(pd.notnull(df), None)
    # Crear el engine de SQLAlchemy
    engine = create_engine(f'postgresql://{usuario}:{password}@{host}/{database}')
        
    # Eliminar todos los registros de la tabla
    with engine.connect() as connection:
        #connection.execute(text(f"DELETE FROM {tabla};"))
        connection.execute(text(f"DELETE FROM {tabla} CASCADE;"))
        connection.commit()

    # Insertar los nuevos datos en la tabla
    df.to_sql(tabla, con=engine, if_exists='append', index=False)  # Solo append en tablas con relaciones    

    # Obtener el valor máximo de la columna 'id'
    with engine.connect() as connection:
        result = connection.execute(text(f"SELECT MAX({columna_id}) FROM {tabla};"))
        max_id = result.scalar() or 0  # Si no hay registros, usar 0

    # Obtener el nombre de la secuencia asociada a la columna 'id'
    with engine.connect() as connection:
        result = connection.execute(text(f"SELECT pg_get_serial_sequence('{tabla}', '{columna_id}');"))
        id_secuencia = result.scalar()

    # Reiniciar el valor de la secuencia si se obtiene la secuencia asociada
    with engine.connect() as connection:
        if id_secuencia:
            connection.execute(text(f"ALTER SEQUENCE {id_secuencia} RESTART WITH {max_id + 1};"))
            connection.commit()
            print(f'Se reinició el índice {id_secuencia} en {max_id + 1}')

# Ejemplo de uso:
## actualizar_tabla_postgres(df_plan, 'plans', 'id')
def obtener_registros( tabla,  usuario='user_mantenimiento', password='pass_M4ntenimient0',host='192.168.100.56', database='db_mantenimiento_test',columna_ids=[]):
    """
    Realiza un SELECT * en una tabla especificada de la base de datos y retorna un DataFrame con los resultados.
    """
    # Crear el engine de SQLAlchemy
    engine = create_engine(f'postgresql://{usuario}:{password}@{host}/{database}')
    
    with engine.connect() as connection:
        # Ejecutar la sentencia SQL para obtener los registros
        result = connection.execute(text(f'select * from {tabla}'))        
        # Convertir los resultados en un DataFrame
        df = pd.DataFrame(result.fetchall(), columns=result.keys())
        #df.index = df['index']
        df.index = df['id']
    return df if not columna_ids or len(columna_ids) == 0 else df[columna_ids]
    
def buscarIndice(df: pd.DataFrame, valor_busqueda:str, columna_busqueda='name'):
    """
    Busca un valor en la columna especificada del DataFrame.
    Si el valor de búsqueda es nulo o si no se encuentra, retorna el mismo valor de búsqueda.
    """
    
    
    # Validar si el valor de búsqueda es nulo
    if pd.isna(valor_busqueda):
        return None #pd.NA    
    # Verificar si el valor está en la columna_id especificada
    resultado = df[df[columna_busqueda].str.contains(valor_busqueda, case=False, na=False)]    
    # Si no encuentra el valor, retornar el mismo valor
    if resultado.empty:
        return valor_busqueda
    else:
        return int(resultado.index[0])

def ejecutar_query(query, usuario='user_mantenimiento', password='pass_M4ntenimient0',host='192.168.100.56', database='db_mantenimiento_test'):
    """
    Ejecuta una consulta SQL y devuelve el resultado en un DataFrame si la consulta devuelve filas.
    """
    # Crear el engine de SQLAlchemy
    engine = create_engine(f'postgresql://{usuario}:{password}@{host}/{database}')
    
    with engine.connect() as connection:
        # Ejecutar la consulta
        result = connection.execute(text(query))
        connection.commit()
        
        # Verificar si la consulta devuelve filas
        if result.returns_rows:
            # Obtener los resultados en un DataFrame
            df = pd.DataFrame(result.fetchall(), columns=result.keys())
            return df
        else:
            # Si no devuelve filas, solo confirmar la ejecución
            return None

    return result
def eliminar_registros(tabla,usuario='user_mantenimiento', password='pass_M4ntenimient0',host='192.168.100.56', database='db_mantenimiento_test'):
    """
    Elimina todos los registros de una tabla especificada en la base de datos.    
    """
    # Crear el engine de SQLAlchemy
    engine = create_engine(f'postgresql://{usuario}:{password}@{host}/{database}')
    
    with engine.connect() as connection:
        # Ejecutar la sentencia SQL para eliminar todos los registros
        connection.execute(text(f"DELETE FROM {tabla};"))
        connection.commit()  # Confirmar los cambios

def __actualizar_tabla_postgres(df:pd.DataFrame , tabla:str, columna_id:str , usuario='postgres', password='postgres', host= 'localhost', database='simyo3'):
    # Crear el engine de SQLAlchemy
    engine = create_engine(f'postgresql://{usuario}:{password}@{host}/{database}')

    # Usar pd.read_sql_query con una conexión
        # Leer la tabla en un DataFrame de pandas
    df_origen = ejecutar_query(f"SELECT * FROM {tabla}")

    
    # Renombrar la columna_id 'value' a 'name' si existe
    if 'value' in df.columns:
        df = df.rename(columns={'value': 'name'})
    
    # Si existe la columna_id uuid en la tabla original, crear esa columna_id
    if 'uuid' in df_origen.columns:
        df['uuid'] = [str(uuid.uuid4()) for _ in range(len(df))]
    
    if 'is_active' in df_origen.columns:
        df['is_active'] = True


    # Identificar las columna_ids que están en df pero no en df_origen
    missing_columns = [col for col in df_origen.columns if col not in df.columns]

    # Añadir las columna_ids faltantes a df con valores NaN
    for col in missing_columns:
        df[col] = None #pd.NA  # O usa otro valor predeterminado si es necesario

    # Añadir las columna_ids comunes en las tablas
    df['id'] = df.index
    
    df['created_by'] = 1
    df['updated_by'] = 1
    df['created_at'] = pd.Timestamp.now()
    df['updated_at'] = pd.Timestamp.now()   

    # Eliminar todos los registros de la tabla
    with engine.connect() as connection:
        connection.execute(text(f"DELETE FROM {tabla};"))
        #connection.commit()
    
    # Insertar nuevos datos con pandas to_sql
    df.to_sql(tabla, con= engine, if_exists='replace', index=False)

    # Obtener el valor máximo de la columna_id 'id'
    with engine.connect() as connection:
        result = connection.execute(text(f"SELECT MAX({columna_id}) FROM {tabla};"))
        max_id = result.scalar() or 0  # Si no hay registros, usar 0

    
    # Obtener el nombre de la secuencia asociada a la columna_id 'id'
    with engine.connect() as connection:
        result = connection.execute(text(f"""
            SELECT pg_get_serial_sequence('{tabla}', '{columna_id}');
        """))
        id_secuencia = result.scalar()  # Obtener el nombre de la secuencia
    
        
    # Reiniciar el valor de la secuencia
    with engine.connect() as connection:
        # Si se tienen 
        if id_secuencia : connection.execute(text(f"ALTER SEQUENCE {id_secuencia} RESTART WITH {max_id + 1};"))

#usuario='user_mantenimiento', password='pass_M4ntenimient0',host='192.168.100.56', database='db_mantenimiento_test'
def update_plans_table(df, tabla, columna_id, usuario='user_mantenimiento', password='pass_M4ntenimient0',host='192.168.100.56', database='db_mantenimiento_test' ):
    # Crear el engine de SQLAlchemy
    engine = create_engine(f'postgresql://{usuario}:{password}@{host}/{database}')

    # Leer la tabla en un DataFrame de pandas
    df = pd.read_sql(f"SELECT * FROM {tabla}", engine)

    # Renombrar columna_id 'value' a 'name' si existe
    if 'value' in df.columns:
        df = df.rename(columns={'value': 'name'})

    # Identificar las columna_ids que están en df pero no en df
    missing_columns = [col for col in df.columns if col not in df.columns]

    # Añadir las columna_ids faltantes a df con valores NaN
    for col in missing_columns:
        df[col] = None  # O usa otro valor predeterminado si es necesario

    # Añadir/actualizar las columna_ids necesarias en df
    df['id'] = df.index
    df['is_active'] = True
    df['created_by'] = 1
    df['updated_by'] = 1
    df['created_at'] = pd.Timestamp.now()
    df['updated_at'] = pd.Timestamp.now()

    # Obtener el nombre de la secuencia asociada a la columna_id 'id' en la tabla 'plans_test'
    with engine.connect() as connection:
        result = connection.execute(text(f"SELECT pg_get_serial_sequence('{tabla}', '{columna_id}');"))
        id_secuencia = result.scalar()  # Obtener el nombre de la secuencia

    # Eliminar todos los registros de la tabla
    with engine.connect() as connection:
        connection.execute(text(f"DELETE FROM {tabla};"))
        connection.commit()

    # Insertar nuevos datos con pandas to_sql
    df.to_sql(tabla, engine, if_exists='replace', index=False)

    # Obtener el valor máximo de la columna_id 'id'
    with engine.connect() as connection:
        result = connection.execute(text(f"SELECT MAX(id) FROM {tabla};"))
        max_id = result.scalar() or 0  # Si no hay registros, usar 0

    # Reiniciar el valor de la secuencia
    with engine.connect() as connection:
        connection.execute(text(f"ALTER SEQUENCE {id_secuencia} RESTART WITH {max_id + 1};"))

# Ejemplo de uso
# Definir el DataFrame df con tus datos

## Pasos
1. Eliminar registros de activities
2. Eliminar de tabla base que este relacionado
3.

In [307]:
#1. Eliminar datos tabla actividades
eliminar_registros(tabla='activities')
# Eliminar registros de la tabla base que tenga items relacionados
query = 'delete from base where fk_plan is not null '
ejecutar_query(query)


In [308]:
df_plan.head(2)

,value
1,MOTOR ELECTRICO PRINCIPAL
2,REFRIGERACION DE MOTOR


In [309]:
# %pip install SQLAlchemy psycopg2-binary

## Tabla actions

In [310]:
# Tabla actions, darle formato y actualizar en BBDD
df_action =format_dataframe(df_action,'actions')
actualizar_tabla_postgres(df_action,'actions','id')

Se reinició el índice public.actions_id_seq en 16


## Tabla Periodicities

In [311]:
df_periodicities = obtener_registros(tabla='periodicities')
df_periodicities

,id,name,is_active,created_by,updated_by,deleted_by,created_at,updated_at,deleted_at
id,,,,,,,,,
1,1,dia(s),True,1,1,None,2022-05-30 12:05:19.768000-04:00,2022-05-30 12:05:19.768000-04:00,None
2,2,semana(s),True,1,1,None,2022-05-30 12:05:19.768000-04:00,2022-05-30 12:05:19.768000-04:00,None
3,3,mes(es),True,1,1,None,2022-05-30 12:05:19.768000-04:00,2022-05-30 12:05:19.768000-04:00,None
4,4,año(s),True,1,1,None,2022-05-30 12:05:19.768000-04:00,2022-05-30 12:05:19.768000-04:00,None
5,5,lunes,True,1,1,None,2022-05-30 12:05:19.768000-04:00,2022-05-30 12:05:19.768000-04:00,None
6,6,martes,True,1,1,None,2022-05-30 12:05:19.768000-04:00,2022-05-30 12:05:19.768000-04:00,None
7,7,miercoles,True,1,1,None,2022-05-30 12:05:19.768000-04:00,2022-05-30 12:05:19.768000-04:00,None
8,8,jueves,True,1,1,None,2022-05-30 12:05:19.768000-04:00,2022-05-30 12:05:19.768000-04:00,None
9,9,viernes,True,1,1,None,2022-05-30 12:05:19.768000-04:00,2022-05-30 12:05:19.768000-04:00,None


## specialties

In [312]:
df_speciality = format_dataframe(df_speciality,'specialties')
df_speciality['description'] = df_speciality['name'] # no se crea automaticamente
actualizar_tabla_postgres(df=df_speciality,tabla='specialties',columna_id='id')

Se reinició el índice public.specialties_id_seq en 9


## plans

In [313]:
# Tabla actions, darle formato y actualizar en BBDD
df_plan = format_dataframe(df_plan,'plans')
actualizar_tabla_postgres(df_plan,'plans','id')

Se reinició el índice public.plans_id_seq en 279


## usage_units

In [314]:
df_usage_units = pd.DataFrame(df_activities['fk_usage_unit'].dropna().unique(),columns=['value'])
df_usage_units = format_dataframe(df_usage_units,'usage_units')
df_usage_units['unit']=df_usage_units['name']           #TODO: ???
df_usage_units['description']=df_usage_units['name']    #TODO: ???
actualizar_tabla_postgres(df_usage_units,'usage_units',columna_id='id')
df_usage_units.index = df_usage_units.id    # Se coloca el id correspondientea

Se reinició el índice public.usage_units_id_seq en 3


In [315]:
df_usage_units

,name,id,created_at,created_by,updated_at,updated_by,deleted_at,deleted_by,unit,description
id,,,,,,,,,,
1,horas,1,2024-10-08 11:26:59.145153,1,2024-10-08 11:26:59.146154,1,None,None,horas,horas
2,ciclos,2,2024-10-08 11:26:59.145153,1,2024-10-08 11:26:59.146154,1,None,None,ciclos,ciclos


## activities

In [316]:
# Obtener registros de la tabla classifiers
df_classifiers = obtener_registros(tabla='classifiers')
df_classifiers.head(2)

,id,fk_classifiers_type,name,description,grup_1,notEnabled,is_active,created_by,updated_by,deleted_by,created_at,updated_at,deleted_at
id,,,,,,,,,,,,,
5,5,3,ACORDE A LA DISPONIBILIDAD,ACORDE A LA DISPONIBILIDAD,1,None,False,1,1.0,1.0,2022-05-30 12:05:18.891000-04:00,2022-05-30 12:05:18.891000-04:00,2022-07-18 10:05:18.891000-04:00
17,17,5,ELECTRICO,ELECTRICO,1,None,True,1,1.0,NaN,2022-05-30 12:05:18.891000-04:00,2022-05-30 12:05:18.891000-04:00,NaT


In [317]:
# Dar formato a tabla
df_activities = format_dataframe(df_activities,'activities')

In [318]:
# Hacer copia local
df_activities_raw = df_activities.copy(deep=True)

In [319]:
df_activities.head(2)

,fk_activity,fk_plan,fk_action,name,fkc_activity_type,fkc_priority,fk_specialty,fkc_regime,stoppage,time_interval_value,...,created_by,updated_at,updated_by,deleted_at,deleted_by,earliest_reschedule_days,latest_reschedule_days,earliest_reschedule_usage,latest_reschedule_usage,skippable
1,None,1,1,Verificar la ausencia de ruidos y vibraciones ...,Actividad,MEDIA,1,FECHAS,False,1.0,...,1,2024-10-08 11:26:59.375324,1,None,None,None,None,None,None,None
2,None,1,1,Verificación de marcas de tornillería del moto...,Actividad,MEDIA,1,FECHAS,False,1.0,...,1,2024-10-08 11:26:59.375324,1,None,None,None,None,None,None,None


In [320]:
# Convertir en sus indices todas las columnas que tengan que ver con classifiers
df_activities['fkc_activity_type'] =df_activities['fkc_activity_type'].apply(lambda x: gs1.buscarIndice(valor=str(x),df=df_classifiers,columna_id='name')) 
df_activities['fkc_priority'] = df_activities['fkc_priority'].apply(lambda x: gs1.buscarIndice(df=df_classifiers,valor=str(x),columna_id='name'))
df_activities['fkc_regime'] = df_activities['fkc_regime'].apply(lambda x: gs1.buscarIndice(df=df_classifiers,valor=str(x),columna_id='name'))

#Obtener periodiciadad unit
df_activities['fk_periodicity_unit'] = df_activities['fk_periodicity_unit'].apply(lambda x: gs1.buscarIndice(valor=str(x),df=df_periodicities,columna_id='name'))
# obtener usage_unit
df_activities['fk_usage_unit'] = df_activities['fk_usage_unit'].apply(lambda x: gs1.buscarIndice(valor=str(x),df=df_usage_units,columna_id='name'))

# en la columna fk_periodicity_unit tienen valores 'NONE', necesito que nos valores 'NONE' se conviertan en valores vacios
df_activities['fk_periodicity_unit'] = df_activities['fk_periodicity_unit'].replace('NONE', None)

# en la columna fk_usage_unit tiene valores "NAN", reemplazar valores NAN con valores vacios o None
df_activities['fk_usage_unit'] = df_activities['fk_usage_unit'].replace('NAN', None)

df_activities.head(2)


,fk_activity,fk_plan,fk_action,name,fkc_activity_type,fkc_priority,fk_specialty,fkc_regime,stoppage,time_interval_value,...,created_by,updated_at,updated_by,deleted_at,deleted_by,earliest_reschedule_days,latest_reschedule_days,earliest_reschedule_usage,latest_reschedule_usage,skippable
1,None,1,1,Verificar la ausencia de ruidos y vibraciones ...,166,8,1,163,False,1.0,...,1,2024-10-08 11:26:59.375324,1,None,None,None,None,None,None,None
2,None,1,1,Verificación de marcas de tornillería del moto...,166,8,1,163,False,1.0,...,1,2024-10-08 11:26:59.375324,1,None,None,None,None,None,None,None


In [321]:
df_activities.to_excel("activities_antes_bd.xlsx")

In [322]:
actualizar_tabla_postgres(df_activities,'activities',columna_id='id')

#InvalidTextRepresentation: la sintaxis de entrada no es válida para tipo integer: «nan»
#LINE 1: ...l motor.', 166, 8, 1, 163, false, 1.0, '1', NULL, 'nan', 'd3...


Se reinició el índice public.activities_id_seq en 2881


## Completar unidades medida y de uso

De df_activities de la columna fk_periodicity_unit, si el valor de la columna es ['horas','ciclos'] mover el contenido de la columna time_interval_value a la columna usage_interval_value en la columna time_interval_value dejar con valor nulo, y mover el valor de la columna fk_peridicity_unit  a la columna fk_usage_unit y en la columna fk_periodicity_unit dejar en nulo 

In [323]:
#%pip install sqlalchemy==1.4.23 psycopg2-binary

In [324]:
#pip install psycopg2-binary

# Instalación
Se colocaron todos los paquetes que se utilizan en el proyecto en el archivo paquetes.txt
En caso que no funcione utilizar:

<code> pip install SQLAlchemy psycopg2-binary </code>



In [325]:
#%pip install SQLAlchemy

## Exportar a archivo excel

In [326]:

#df_plan, df_action, df_speciality, filtered_data = gs1.process_data(filename=filename,valores=valores,regimen=regimen)      
######################    

#####################
#with pd.ExcelWriter("output/"+salida.value) as writer:
#   df_action.to_excel(writer, sheet_name='actions')
#    df_plan.to_excel(writer, sheet_name='plans')
#    df_speciality.to_excel(writer, sheet_name='specialties')
#    df_activities.to_excel(writer, sheet_name='activities')

# gs1.save_to_excel(output_path=salida.value,valores=valores,regimen=regimen,filename="mix_plan.csv")        


In [327]:
''' # https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
import warnings
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Button, Layout
from IPython.display import display

#warnings.simplefilter(action='ignore', category=FutureWarning)

valores = {
    "D": 1,
    "S": 1,
    "2S": 2,
    "M": 5,
    "MC": 1,
    "2M": 2,
    "T": 3,
    "SE": 6,
    "8M": 8,
    "A": 1,
    "1.5A": 18,
    "2A": 2,
    "3A": 3,
    "4A": 4,
    "5A": 5,
    "6A": 6,
    "8A": 8,
    "10A": 10,
    "1000": 1000,
    "1300": 1300,
    "1800": 1800,
    "6000": 6000,
    "22500": 6000,
    "40000": 40000,
    "55000": 55000,
    "55000C": 55000
}

regimen = {
    "D": 'dia',
    "S": 'semana',
    "2S": 'semana',
    "M": 'semana',
    "MC": 'mes',
    "2M": 'mes',
    "T": 'mes',
    "SE": 'mes',
    "8M": 'mes',
    "A": 'Año',
    "1.5A": 'mes',
    "2A": 'Año',
    "3A": 'Año',
    "4A": 'Año',
    "5A": 'Año',
    "6A": 'Año',
    "8A": 'Año',
    "10A": 'Año',
    "1000": 'horas',
    "1300": 'horas',
    "1800": 'horas',
    "6000": 'horas',
    "22500": 'horas',
    "40000": 'horas',
    "55000": 'horas',
    "55000C": 'ciclos'
}

lad = widgets.Textarea(value='https://docs.google.com/spreadsheets/d/1oUHkuKpHtuhMirNW6SvAQ4A0ns5PZs71iZ_WFXZHNn8/edit?gid=1199302294',placeholder='Plan Maestro LAD',description='Lineas Alta Demanda:',disabled=False,layout=Layout(width='70%',height="200px"))
lbd = widgets.Textarea(value='https://docs.google.com/spreadsheets/d/1yOaSeqRBr1FW6tvFMi_Y-s4011cKBoyiWU5dTMlujrU/edit?gid=1199302294',placeholder='Plan Maestro LBD',description='Lineas Baja Demanda:',disabled=False,layout=Layout(width='70%',height="200px"))
host = widgets.Text(value='192.168.100.50',placeholder='Host',description='Host:',disabled=False)
basedatos = widgets.Text(value='simyo2',placeholder='BaseDatos',description='BaseDatos',disabled=False)
usuario = widgets.Text(value='mantto',description='Usuario')
password = widgets.Password(value='Sistemas0',description='Password')
button1 = widgets.Button(description="Generar Archivo Excel",button_style='success',layout=Layout(width='20%'))
button2 = widgets.Button(description="Cargar en Base de datos",button_style='danger',layout=Layout(width='20%'))
output = widgets.Output()
salida = widgets.Text(value="Salida.xlsx",description="Nombre:",disabled=False)
accordion = widgets.Accordion(children=[ salida], titles=(['Archivo Salida']))
accordion1 = widgets.Accordion(children=[ usuario,password,host,basedatos], titles=('Usuario','Password','Host','Base de Datos'))

display(lad,lbd,host,usuario,password,accordion,button1, accordion1,button2,output)

def on_button_clicked(b):    
    # Combinacion de dataframes
    gs1 = GoogleSheetProcessor(lad.value) #("https://docs.google.com/spreadsheets/d/1oUHkuKpHtuhMirNW6SvAQ4A0ns5PZs71iZ_WFXZHNn8/edit?gid=1199302294")
    archivo = "input/pad.csv"
    #gs.download_csv(archivo)
    df1 = gs1.read_csv(archivo)

    gs2 = GoogleSheetProcessor (lbd.value) #("https://docs.google.com/spreadsheets/d/1yOaSeqRBr1FW6tvFMi_Y-s4011cKBoyiWU5dTMlujrU/edit?gid=1199302294")
    archivo = "input/pbd.csv"
    #gs.download_csv(archivo)
    df2 = gs2.read_csv(archivo)
    # Realizar el merge de ambos planes en un solo dataframe
    df_merged = pd.concat([df1,df2],ignore_index=True).reset_index(drop=True)

    filename = "input/mix_plan.csv"
    df_merged.to_csv(filename)
    
    #df_plan, df_action, df_speciality, filtered_data = gs1.process_data(filename=filename,valores=valores,regimen=regimen)      
######################    

#####################

    df_plan, df_action, df_speciality, filtered_data = gs1.process_data(filename=filename,valores=valores,regimen=regimen)  
    with pd.ExcelWriter("output/"+salida.value) as writer:
        df_action.to_excel(writer, sheet_name='actions')
        df_plan.to_excel(writer, sheet_name='plans')
        df_speciality.to_excel(writer, sheet_name='specialties')
        filtered_data.to_excel(writer, sheet_name='activities')
    
    # gs1.save_to_excel(output_path=salida.value,valores=valores,regimen=regimen,filename="mix_plan.csv")        
    with output:
        print("Se Genera archivo excel Salida.xlsx")

button1.on_click(on_button_clicked)
button2.on_click(lambda _: print("Boton 2 accionado"))
#https://ipywidgets.readthedocs.io/en/7.6.3/examples/Widget%20Styling.html '''

' # https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html\nimport warnings\nimport pandas as pd\nimport ipywidgets as widgets\nfrom ipywidgets import Button, Layout\nfrom IPython.display import display\n\n#warnings.simplefilter(action=\'ignore\', category=FutureWarning)\n\nvalores = {\n    "D": 1,\n    "S": 1,\n    "2S": 2,\n    "M": 5,\n    "MC": 1,\n    "2M": 2,\n    "T": 3,\n    "SE": 6,\n    "8M": 8,\n    "A": 1,\n    "1.5A": 18,\n    "2A": 2,\n    "3A": 3,\n    "4A": 4,\n    "5A": 5,\n    "6A": 6,\n    "8A": 8,\n    "10A": 10,\n    "1000": 1000,\n    "1300": 1300,\n    "1800": 1800,\n    "6000": 6000,\n    "22500": 6000,\n    "40000": 40000,\n    "55000": 55000,\n    "55000C": 55000\n}\n\nregimen = {\n    "D": \'dia\',\n    "S": \'semana\',\n    "2S": \'semana\',\n    "M": \'semana\',\n    "MC": \'mes\',\n    "2M": \'mes\',\n    "T": \'mes\',\n    "SE": \'mes\',\n    "8M": \'mes\',\n    "A": \'Año\',\n    "1.5A": \'mes\',\n    "2A": \'Año\',\n    "3A": \'Año\',\n

In [328]:
# Obtener las columnas y sus tipos de datos de la columna activities
#query = """
#SELECT column_name
#FROM information_schema.columns
#WHERE table_name = 'activities' order by ordinal_position asc;
#"""
#df_columnas_activities = ejecutar_query(query)
#df_columnas_activities['column_name']
